# Terrestrial Laser Scanning

This notebook is designed to take terrestrial laser scanner (TLS) data from the SnowEx Alaska Campaigns and derive snow depth. The TLS data is provided in both a raw point cloud format and a processed DEM format. For this example, we will be focusing on the TLS DEMs.

The TLS data is available through the cloud on NSIDC, so we will be using the `earthaccess` package. The first example will involve a single TLS image for simplicity, then we will have a second example that examines multiple TLS scans from the campaigns.

In [ ]:
import earthaccess
import matplotlib.pyplot as plt
import numpy as np
import re
import rioxarray as rxr
import shutil
import tempfile
import xarray as xr

The TLS data was gathered in Bonanza Creek near Fairbanks, AK in two months: October 2022 and March 2023. These months correspond to the snow-off and snow-on seasons, respectively. We will start by getting some sample snow-on TLS data from a single day.

In [ ]:
# Authenticate with Earthdata Login servers
auth = earthaccess.login(strategy="interactive")

# Search for snow-on granules
results = earthaccess.search_data(
    #short_name="SNEX23_BCEF_TLS",
    doi = "10.5067/R466GRXNA61S",
    temporal=('2023-03-15', '2023-03-15'),
)

Because the TLS data is available on-demand through the cloud, we do not need to download it. Instead, we can stream it directly with `rioxarray`!

In [ ]:
# Load a single TLS scan
files = earthaccess.open(results)
snow_on = rxr.open_rasterio(files[1])

In [ ]:
snow_on.rio.width

In [ ]:
# Visualize the snow-on data
fig, ax = plt.subplots()
snow_on.plot(ax=ax, vmin=123, vmax=126,
             cbar_kwargs={'label': "Elevation [m]"})
ax.set_xlabel("Easting [m]")
ax.set_ylabel("Northing [m]")
ax.set_title(" ")

Two things are noticeable from this TLS data:
1. It has a very high resolution (0.15 m).
2. The signal attenutates after ~60 m, so we have a small field of view.

This suggests that we will be able to obtain very fine-scale measurements of snow depth, but we will need scans from multiple locations to better characterize snow in Bonanza Creek.

In any case, let's grab the snow-off data from the same location, and try to derive snow depth.

In [ ]:
# Now search for snow-off granules
results = earthaccess.search_data(
    #short_name="SNEX23_BCEF_TLS",
    doi = "10.5067/R466GRXNA61S",
    temporal=('2022-10-25', '2022-10-25'),
)

In [ ]:
display(results)

In [ ]:
# Again, load a single snow-off TLS scan
files = earthaccess.open(results)
snow_off = rxr.open_rasterio(files[1])

In [ ]:
fig, ax = plt.subplots()
snow_off.plot(vmin=123, vmax=126,
              cbar_kwargs={'label': "Elevation [m]"})
ax.set_xlabel("Easting [m]")
ax.set_ylabel("Northing [m]")
ax.set_title(" ")

Although the snow-on/-off data look similar to each other, there are slight differences, meaning that we cannot perform a difference right away. We must first interpolate the data, ensuring that fill values are accounted for, then perform the difference.

In [ ]:
# Interpolate snow-on data onto the x/y grid of snow-off data
snow_on_interp = snow_on.interp(
    x=snow_off.x,
    y=snow_off.y,
    kwargs={"fill_value": snow_on.attrs.get('_FillValue', np.nan)}
)

# Calculate the difference (snow depth)
difference = snow_on_interp - snow_off

# Define fill values in data
fill = snow_off.attrs.get('_FillValue', -9999.0)

# Include only data that is not equal to the fill value
difference = difference.where((snow_off != fill) & (snow_on_interp != fill))

In [ ]:
# Plot snow depth over the TLS scene
fig, ax = plt.subplots()
difference.plot(vmin=0, vmax=1.5,
                cbar_kwargs={'label': "Snow depth [m]"})
ax.set_xlabel("Easting [m]")
ax.set_ylabel("Northing [m]")
ax.set_title(" ")

Although not perfect, this provides a very reasonable snow depth DEM for the TLS data gathered in this location. If we want, we can perform basic statistics on the derived snow depths.

In [ ]:
# Calculate median snow depth over the scene
median_depth = difference.where(difference>=0).median()

# Make histogram plot of snow depth
fig, ax = plt.subplots()
difference.where(difference>=0).plot.hist(ax=ax, bins=50)
ax.axvline(x=median_depth, color='black', linewidth=2, linestyle='--') # Median depth line
ax.set_xlim([0, 2.5])
ax.set_ylabel("Counts")
ax.set_xlabel("Snow depth [m]")
ax.set_title(' ')
ax.text(1, 8000, f'Median depth = {median_depth:.2f} m', fontsize=12)

# Multiple Scans Example

Because we can stream the TLS data through the cloud, this example is very similar to the above code. The main exception is that we will generate a list of DataArrays, from which we derive snow depth for three TLS scanning locations.

In [ ]:
# Search for snow-on granules
snow_on_results = earthaccess.search_data(
    #short_name="SNEX23_BCEF_TLS",
    doi = "10.5067/R466GRXNA61S",
    temporal=('2023-03-01', '2023-03-31'),
)

snow_off_results = earthaccess.search_data(
    #short_name="SNEX23_BCEF_TLS",
    doi = "10.5067/R466GRXNA61S",
    temporal=('2022-10-01', '2022-10-31'),
)

In [ ]:
# Create list of snow-on DataArrays
snow_on_files = earthaccess.open(snow_on_results)
snow_on_rasters = [rxr.open_rasterio(f) for f in snow_on_files]

# Create list of snow-off DataArrays
snow_off_files = earthaccess.open(snow_off_results)
snow_off_rasters = [rxr.open_rasterio(f) for f in snow_off_files]

To make the final plot of this example cleaner, we will assign each TLS scan a label based on the site ID at Bonanza Creek.

In [ ]:
snon_site_ids = []
snoff_site_ids = []
# Get site IDs for each snow-on DataArray
for f in snow_on_files:
    # Get path from file name
    path = f.path
    # Use regex to extract the site ID from file path, given pattern _SW_YYYYMMDD_SITEID_V
    m = re.search(r'_(SW|N)_\d{8}_(.*?)_V', path)
    if m:
        snon_site_ids.append(m.group(2))
    else:
        snon_site_ids.append("unknown")

# Get site IDs for each snow-off DataArray
for f in snow_off_files:
    # Step 1: Extract path
    path = f.path
    # Step 2: Use regex to extract the site ID
    # Pattern: _SW_YYYYMMDD_SITEID_V
    m = re.search(r'_(SW|N|NE)_\d{8}_(.*?)_V', path)
    if m:
        snoff_site_ids.append(m.group(2))
    else:
        snoff_site_ids.append("unknown")

print(snon_site_ids)
print(snoff_site_ids)

In [ ]:
# Add site ID to attributes of DataArrays
for r, site in zip(snow_on_rasters, snon_site_ids):
    r.attrs['site_id'] = site

for r, site in zip(snow_off_rasters, snoff_site_ids):
    r.attrs['site_id'] = site

In [ ]:
# Create dictionaries linking each DataArray to a site ID
snow_on_dict = {r.attrs['site_id']: r for r in snow_on_rasters}
snow_off_dict = {r.attrs['site_id']: r for r in snow_off_rasters}

Now each TLS scan is linked to a site ID. However, we can see that the snow-on data has many more scans than the snow-off data. Because snow depth data is our priority, we will only consider snow-on scans that share a site ID with the snow-off data.

In [ ]:
# Determine site IDs with recorded data for both snow-off and snow-on season
common_site_ids = sorted(set(snow_on_dict).intersection(snow_off_dict))
print("Common site IDs:", common_site_ids)

In [ ]:
# Create lists of DataArrays for the common sites only
snow_on_paired = [snow_on_dict[sid] for sid in common_site_ids]
snow_off_paired = [snow_off_dict[sid] for sid in common_site_ids]

Now that the site IDs are matched, deriving snow depth is the same as the first example, only with looping to make the calculation (and plotting) easier.

In [ ]:
snow_depths = []
# Interpolate DataArrays and derive snow depth, as before
for so, soff, site in zip(snow_on_paired, snow_off_paired, common_site_ids):
    # Interpolate snow-on data onto the x/y grid of snow-off data
    tmp_interp = so.interp(
        x=soff.x,
        y=soff.y,
    )

    tmp_diff = tmp_interp - soff
    tmp_diff.attrs['site_id'] = site

    tmp_diff = tmp_diff.where((tmp_diff[0]>0)&(tmp_diff[0]<=2))
    snow_depths.append(tmp_diff)

In [ ]:
# Plot the derived snow depths in a 3x3 figure
fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.flatten()

for idx, data_array in enumerate(snow_depths):
    data_array.plot(ax=axes[idx], vmin=0, vmax=2)
    axes[idx].set_title(f"{snow_depths[idx].attrs['site_id']}")

plt.tight_layout()
plt.show()

That's all there is to it! Some of the coverage is a bit sparse, and the depths over site DEC look rather high, but we otherwise have reasonable snow depths over 9 sites in Bonanza Creek. These could then be compared to other ground based efforts or airborne data to cross-calibrate observation methods.